In [1]:
import os, sys
import warnings
import random
import time

from vizdoom import *

import numpy as np
from skimage import transform, util, color
from collections import deque
import matplotlib.pyplot as plt

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [3]:
%cd ..\..
%matplotlib inline
warnings.filterwarnings('ignore')

C:\Users\ethan\OneDrive\notebooks


In [4]:
### MODEL HYPERPARAMETERS (from article) ###
im_size = [100,100,4]        # 200x200 image x 4 frames per input
action_size = 7              # 
learning_rate =  0.00025      # Alpha (aka learning rate)

### TRAINING HYPERPARAMETERS
total_episodes = 5000         # Total episodes for training
max_steps = 5000              # Max possible steps in an episode
batch_size = 64             
save_frequency = 1000

# FIXED Q TARGETS HYPERPARAMETERS 
max_tau = 10000 #Tau is the C step where we update our target network

# EXPLORATION HYPERPARAMETERS for epsilon greedy strategy
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.00005            # exponential decay rate for exploration prob

# Q LEARNING hyperparameters
gamma = 0.95               # Discounting rate

### MEMORY HYPERPARAMETERS
## If you have GPU change to 1million
# pretrain_length = 100000   # Number of experiences stored in the Memory when initialized for the first time
pretrain_length = 10000
memory_size = 10000       # Number of experiences the Memory can keep

### MODIFY THIS TO FALSE IF YOU JUST WANT TO SEE THE TRAINED AGENT
training = False

## TURN THIS TO TRUE IF YOU WANT TO RENDER THE ENVIRONMENT
episode_render = True
trained_model = 'DQNcheckpoint1000.pth.tar'

In [5]:
def createEnvironment():
    game = DoomGame()
    game.load_config("scenarios/deadly_corridor.cfg")
    game.init()
    
    possible_actions = np.identity(7,dtype=int).tolist()
    
    return game, possible_actions

In [6]:
def processImg(img):
#     img = np.swapaxes(img, 0, 2)
#     img = color.rgb2gray(img)
    img = img[45:-10, :]
    img = img / 255
    img = transform.resize(img, (im_size[0], im_size[1]))
    return img

game, actions = createEnvironment()
img = processImg(game.get_state().screen_buffer)
img.shape
plt.imshow(img)
plt.show()

In [7]:
stack_size = 3
stacked_frames = deque([np.zeros((im_size[0],im_size[1]), dtype=np.int) for i in range(stack_size)], maxlen=3)
def stackFrames(stacked_frames, stateim, is_new_episode):
    frame = processImg(stateim)
    if is_new_episode:
        stacked_frames = deque([np.zeros((im_size[0],im_size[1]), dtype=np.int) for i in range(stack_size)], maxlen=3)
        for _ in range(stack_size):
            stacked_frames.append(frame)
        stacked_state = np.stack(stacked_frames, axis=2)
    else:
        stacked_frames.append(frame)
        stacked_state = np.stack(stacked_frames, axis=2)
        
    return stacked_state, stacked_frames

In [8]:
class D3QN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        #State value
        self.v_fc1 = nn.Linear(12*12*128, 512)
        self.v_fc2 = nn.Linear(512, 1)
        
        #Action
        self.a_fc1 = nn.Linear(12*12*128, 512)
        self.a_fc2 = nn.Linear(512, 7)
    def forward(self, x):
        x = F.elu(self.bn1(self.conv1(x)))
        x = F.elu(self.bn2(self.conv2(x)))
        x = F.elu(self.bn3(self.conv3(x)))
        
        x = x.view(x.size(0), -1)
        v = F.elu(self.v_fc1(x))
        v = F.elu(self.v_fc2(v))
        
        a = F.elu(self.a_fc1(x))
        a = F.elu(self.a_fc2(a))
        out = v + (a-torch.mean(a, 1, True))
        return out

In [9]:
class NNet():
    def __init__(self):
        self.nnet = D3QN().cuda()
        
    def __call__(self):
        return self.nnet

    def train(self, memory, batch_size, qNetwork): 
        optimizer = optim.Adam(self.nnet.parameters(), lr=learning_rate)
        
        tree_idx, batch, ISWeights_mb = memory.sample(batch_size)
        
        states_mb = torch.FloatTensor([each[0][0] for each in batch]).contiguous().cuda()
        actions_mb = torch.FloatTensor([each[0][1] for each in batch]).contiguous().cuda()
        rewards_mb = torch.FloatTensor([each[0][2] for each in batch]).contiguous().cuda()
        next_states_mb = torch.FloatTensor([each[0][3] for each in batch]).contiguous().cuda()
        dones_mb = np.array([each[0][4] for each in batch])
        target_Qs_batch = []
        
        Qs_next_state = qNetwork(next_states_mb.transpose(1,3))
        q_target_next_state = self.nnet(next_states_mb.transpose(1,3))
        
            
        for i in range(0, len(batch)):
            terminal = dones_mb[i]
            action = torch.argmax(Qs_next_state[i])
            if terminal:
                target_Qs_batch.append(rewards_mb[i])
            else:
#                 print(Qs_next_state.shape)
#                 print(torch.max(Qs_next_state[i], 0))
#                 print(torch.max(Qs_next_state[i], 1))
                Q_target = rewards_mb[i] + gamma*q_target_next_state[i][action]
                target_Qs_batch.append(Q_target)
        
        targets_mb = torch.FloatTensor([each for each in target_Qs_batch]).contiguous().cuda()
        ISWeights_mb = torch.FloatTensor(ISWeights_mb).contiguous().cuda()
        
        states_mb, targets_mb, ISWeights_mb = Variable(states_mb), Variable(targets_mb), Variable(ISWeights_mb)
        
        x = self.nnet(states_mb.transpose(1,3))
        loss, abs_error = self.loss(targets_mb, x, ISWeights_mb, actions_mb)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        return tree_idx, loss, abs_error
    
    def predict(self, frames):
        frames = torch.FloatTensor(frames)
        frames = frames.contiguous().cuda().unsqueeze(0)
        with torch.no_grad():
            frames = Variable(frames)
        self.nnet.eval()
        x = self.nnet(frames.transpose(1,3))
        return x.data.cpu().numpy()[0]
    
    def load_weights(self, Nnet):
        self.nnet.load_state_dict(Nnet.state_dict())
    
    def loss(self, target_Qs, output, ISWeights_, actions_):
        Qs = torch.sum(output*actions_, dim=1)
        with torch.no_grad():
            absolute_errors = torch.abs(target_Qs-Qs)
#         loss = (target_Qs-Qs)**2
        return torch.mean(ISWeights_ * (target_Qs-Qs)**2), absolute_errors

    def save_checkpoint(self, folder='checkpoint', filename='checkpoint.pth.tar'):
        filepath = os.path.join(folder, filename)
        if not os.path.exists(folder):
            print("Checkpoint Directory does not exist! Making directory {}".format(folder))
            os.mkdir(folder)
        else:
            print("Saving Checkpoint")
        torch.save({
            'state_dict' : self.nnet.state_dict(),
        }, filepath)
        
    def load_checkpoint(self, folder='checkpoint', filename='checkpoint.pth.tar'):
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py#L98
        filepath = os.path.join(folder, filename)
        if not os.path.exists(filepath):
            raise("No model in path {}".format(filepath))
        checkpoint = torch.load(filepath)
        self.nnet.load_state_dict(checkpoint['state_dict'])
        
    @property
    def state_dict(self):
        return self.nnet.state_dict()

In [10]:
DQNetwork = NNet()
TargetNetwork = NNet()

class SumTree():
    data_pointer = 0
    def __init__(self, capacity):
        self.capacity = capacity
        self.tree = np.zeros(2 * capacity - 1)
        self.data = np.zeros(capacity, dtype=object)
        
    def add(self, priority, data):
        tree_index = self.data_pointer + self.capacity - 1
        
        self.data[self.data_pointer] = data
        self.update(tree_index, priority)
        self.data_pointer += 1
        if self.data_pointer >= self.capacity:
            self.data_pointer = 0
            
    def update(self, tree_index, priority):
        change = priority - self.tree[tree_index]
        self.tree[tree_index] = priority
        
        while tree_index != 0:
            tree_index = (tree_index-1) // 2
            self.tree[tree_index] += change
    def get_leaf(self, v):
        parent_index = 0
        while True:
            left_child_index = 2 * parent_index + 1
            right_child_index = left_child_index + 1
            
            if left_child_index >= len(self.tree):
                leaf_index = parent_index
                break
                
            else:
                if v <= self.tree[left_child_index]:
                    parent_index = left_child_index
                    
                else:
                    v -= self.tree[left_child_index]
                    parent_index = right_child_index
                    
        data_index = leaf_index - self.capacity - 1
        return leaf_index, self.tree[leaf_index], self.data[data_index]
    
    @property
    def total_priority(self):
        return self.tree[0] #return root node

class Memory():
    PAR_e = 0.01
    PAR_a = 0.6
    PAR_b = 0.4
    PAR_b_increment = 0.001
    abs_error_clip = 1.
    
    def __init__(self, capacity):
        self.tree = SumTree(capacity)
    
    def add(self, experience):
        max_priority = np.max(self.tree.tree[-self.tree.capacity:])
        if max_priority == 0:
            max_priority = self.abs_error_clip
        self.tree.add(max_priority, experience)
    
    def sample(self, n):
        memory_b = []
        
        b_idx, b_ISWeights = np.empty((n,), dtype=np.int32), np.empty((n, 1), dtype=np.float32)
        
        priority_segment = self.tree.total_priority / n #divide priority into ranges
        self.PAR_b = np.min([1., self.PAR_b + self.PAR_b_increment]) #increasing b each sampling
        
        p_min = np.min(self.tree.tree[-self.tree.capacity:]) / self.tree.total_priority
        max_weight = (p_min * n) ** (-self.PAR_b) #(1/n * 1/p)^b
        
        for i in range(n):
            a, b = priority_segment * i, priority_segment * (i + 1)
            value = np.random.uniform(a, b)
            
            index, priority, data = self.tree.get_leaf(value)
            
            sampling_probabilities = priority / self.tree.total_priority #P(j)
            
            #IS = (1/N * 1/P(i))^b /max wi == (N*P(i))**-b  /max wi
            b_ISWeights[i, 0] = np.power(n * sampling_probabilities, -self.PAR_b)/ max_weight
            b_idx[i] = index
            experience = [data]
            memory_b.append(experience)
            
        return b_idx, memory_b, b_ISWeights
    
    def batch_update(self, tree_idx, abs_errors):
        abs_errors += self.PAR_e  # convert to abs and avoid 0
        clipped_errors = np.minimum(abs_errors, self.abs_error_clip)
        ps = np.power(clipped_errors, self.PAR_a)

        for ti, p in zip(tree_idx, ps):
            self.tree.update(ti, p)

In [11]:
class SumTree(object):
    """
    This SumTree code is modified version of Morvan Zhou: 
    https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/5.2_Prioritized_Replay_DQN/RL_brain.py
    """
    data_pointer = 0
    
    """
    Here we initialize the tree with all nodes = 0, and initialize the data with all values = 0
    """
    def __init__(self, capacity):
        self.capacity = capacity # Number of leaf nodes (final nodes) that contains experiences
        
        # Generate the tree with all nodes values = 0
        # To understand this calculation (2 * capacity - 1) look at the schema above
        # Remember we are in a binary node (each node has max 2 children) so 2x size of leaf (capacity) - 1 (root node)
        # Parent nodes = capacity - 1
        # Leaf nodes = capacity
        self.tree = np.zeros(2 * capacity - 1)
        
        """ tree:
            0
           / \
          0   0
         / \ / \
        0  0 0  0  [Size: capacity] it's at this line that there is the priorities score (aka pi)
        """
        
        # Contains the experiences (so the size of data is capacity)
        self.data = np.zeros(capacity, dtype=object)
    
    
    """
    Here we add our priority score in the sumtree leaf and add the experience in data
    """
    def add(self, priority, data):
        # Look at what index we want to put the experience
        tree_index = self.data_pointer + self.capacity - 1
        
        """ tree:
            0
           / \
          0   0
         / \ / \
tree_index  0 0  0  We fill the leaves from left to right
        """
        
        # Update data frame
        self.data[self.data_pointer] = data
        
        # Update the leaf
        self.update (tree_index, priority)
        
        # Add 1 to data_pointer
        self.data_pointer += 1
        
        if self.data_pointer >= self.capacity:  # If we're above the capacity, you go back to first index (we overwrite)
            self.data_pointer = 0
            
    
    """
    Update the leaf priority score and propagate the change through tree
    """
    def update(self, tree_index, priority):
        # Change = new priority score - former priority score
        change = priority - self.tree[tree_index]
        self.tree[tree_index] = priority
        
        # then propagate the change through tree
        while tree_index != 0:    # this method is faster than the recursive loop in the reference code
            
            """
            Here we want to access the line above
            THE NUMBERS IN THIS TREE ARE THE INDEXES NOT THE PRIORITY VALUES
            
                0
               / \
              1   2
             / \ / \
            3  4 5  [6] 
            
            If we are in leaf at index 6, we updated the priority score
            We need then to update index 2 node
            So tree_index = (tree_index - 1) // 2
            tree_index = (6-1)//2
            tree_index = 2 (because // round the result)
            """
            tree_index = (tree_index - 1) // 2
            self.tree[tree_index] += change
    
    
    """
    Here we get the leaf_index, priority value of that leaf and experience associated with that index
    """
    def get_leaf(self, v):
        """
        Tree structure and array storage:
        Tree index:
             0         -> storing priority sum
            / \
          1     2
         / \   / \
        3   4 5   6    -> storing priority for experiences
        Array type for storing:
        [0,1,2,3,4,5,6]
        """
        parent_index = 0
        
        while True: # the while loop is faster than the method in the reference code
            left_child_index = 2 * parent_index + 1
            right_child_index = left_child_index + 1
            
            # If we reach bottom, end the search
            if left_child_index >= len(self.tree):
                leaf_index = parent_index
                break
            
            else: # downward search, always search for a higher priority node
                
                if v <= self.tree[left_child_index]:
                    parent_index = left_child_index
                    
                else:
                    v -= self.tree[left_child_index]
                    parent_index = right_child_index
            
        data_index = leaf_index - self.capacity + 1

        return leaf_index, self.tree[leaf_index], self.data[data_index]
    
    @property
    def total_priority(self):
        return self.tree[0] # Returns the root node

In [12]:
class Memory(object):  # stored as ( s, a, r, s_ ) in SumTree
    """
    This SumTree code is modified version and the original code is from:
    https://github.com/jaara/AI-blog/blob/master/Seaquest-DDQN-PER.py
    """
    PER_e = 0.01  # Hyperparameter that we use to avoid some experiences to have 0 probability of being taken
    PER_a = 0.6  # Hyperparameter that we use to make a tradeoff between taking only exp with high priority and sampling randomly
    PER_b = 0.4  # importance-sampling, from initial value increasing to 1
    
    PER_b_increment_per_sampling = 0.001
    
    absolute_error_upper = 1.  # clipped abs error

    def __init__(self, capacity):
        # Making the tree 
        """
        Remember that our tree is composed of a sum tree that contains the priority scores at his leaf
        And also a data array
        We don't use deque because it means that at each timestep our experiences change index by one.
        We prefer to use a simple array and to overwrite when the memory is full.
        """
        self.tree = SumTree(capacity)
        
    """
    Store a new experience in our tree
    Each new experience have a score of max_prority (it will be then improved when we use this exp to train our DDQN)
    """
    def add(self, experience):
        # Find the max priority
        max_priority = np.max(self.tree.tree[-self.tree.capacity:])
        
        # If the max priority = 0 we can't put priority = 0 since this exp will never have a chance to be selected
        # So we use a minimum priority
        if max_priority == 0:
            max_priority = self.absolute_error_upper
        
        self.tree.add(max_priority, experience)   # set the max p for new p

        
    """
    - First, to sample a minibatch of k size, the range [0, priority_total] is / into k ranges.
    - Then a value is uniformly sampled from each range
    - We search in the sumtree, the experience where priority score correspond to sample values are retrieved from.
    - Then, we calculate IS weights for each minibatch element
    """
    def sample(self, n):
        # Create a sample array that will contains the minibatch
        memory_b = []
        
        b_idx, b_ISWeights = np.empty((n,), dtype=np.int32), np.empty((n, 1), dtype=np.float32)
        
        # Calculate the priority segment
        # Here, as explained in the paper, we divide the Range[0, ptotal] into n ranges
        priority_segment = self.tree.total_priority / n       # priority segment
    
        # Here we increasing the PER_b each time we sample a new minibatch
        self.PER_b = np.min([1., self.PER_b + self.PER_b_increment_per_sampling])  # max = 1
        
        # Calculating the max_weight
        p_min = np.min(self.tree.tree[-self.tree.capacity:]) / self.tree.total_priority
        max_weight = (p_min * n) ** (-self.PER_b)
        
        for i in range(n):
            """
            A value is uniformly sample from each range
            """
            a, b = priority_segment * i, priority_segment * (i + 1)
            value = np.random.uniform(a, b)
            
            """
            Experience that correspond to each value is retrieved
            """
            index, priority, data = self.tree.get_leaf(value)
            
            #P(j)
            sampling_probabilities = priority / self.tree.total_priority
            
            #  IS = (1/N * 1/P(i))**b /max wi == (N*P(i))**-b  /max wi
            b_ISWeights[i, 0] = np.power(n * sampling_probabilities, -self.PER_b)/ max_weight
                                   
            b_idx[i]= index
            
            experience = [data]
            
            memory_b.append(experience)
        
        return b_idx, memory_b, b_ISWeights
    
    """
    Update the priorities on the tree
    """
    def batch_update(self, tree_idx, abs_errors):
        abs_errors += self.PER_e  # convert to abs and avoid 0
        clipped_errors = np.minimum(abs_errors, self.absolute_error_upper)
        ps = np.power(clipped_errors, self.PER_a)

        for ti, p in zip(tree_idx, ps):
            self.tree.update(ti, p)

In [13]:
game, possible_actions = createEnvironment()

In [14]:
memory = Memory(memory_size)
game.new_episode()

if training:
    for i in range(pretrain_length):
        if i == 0:
            state = game.get_state().screen_buffer
            state, stacked_frames = stackFrames(stacked_frames, state, is_new_episode=True)

        action = random.choice(possible_actions)
        reward = game.make_action(action)
        done = game.is_episode_finished()

        if done:
            next_state = np.zeros(state.shape)
            memory.add((state, action, reward, next_state, done))

            game.new_episode()
            state = game.get_state().screen_buffer
            state, stacked_frames = stackFrames(stacked_frames, state, is_new_episode=True)
        else:
            next_state = game.get_state().screen_buffer
            next_state, stacked_frames = stackFrames(stacked_frames, next_state, is_new_episode=False)
            memory.add((state, action, reward, next_state, done))
            state = next_state

In [15]:
def predict(explore_start, explore_stop, decay_rate, decay_step, state, possible_actions):
    exp_exp_tradeoff = np.random.rand()
    
    explore_probability = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * decay_step)
    if explore_probability > exp_exp_tradeoff:
        action = random.choice(possible_actions)
    else:
        Qs = DQNetwork.predict(state)
        choice = np.argmax(Qs)
        action = possible_actions[int(choice)]
        
    return action, explore_probability

In [16]:
if training:
    decay_step = 0
    tau = 0
    
    game.init()
    TargetNetwork.load_weights(DQNetwork())
    
    for episode in range(total_episodes):
        if episode > 0:
            print(f"Episode: {episode}, Total Reward: {np.sum(episode_rewards)}, Loss: {loss}, Explore: {exploration_probability}")
        step = 0
        episode_rewards = []
        game.new_episode()
        state = game.get_state().screen_buffer
        state, stacked_frames = stackFrames(stacked_frames, state, True)
        while step < max_steps:
            step += 1
            tau += 1
            decay_step += 1

            action, exploration_probability = predict(explore_start, explore_stop, decay_rate, decay_step, state, possible_actions)
            reward = game.make_action(action)
            done = game.is_episode_finished()
            episode_rewards.append(reward)
            if done:
                next_state = np.zeros((im_size[0], im_size[1]), dtype=np.int)
                next_state, stacked_frames = stackFrames(stacked_frames, next_state, False)
                step = max_steps
                total_reward = np.sum(episode_rewards)
    #             print(f"Episode: {episode}, Total Reward: {total_reward}, Loss: {loss}, Explore: {exploration_probability}")
                memory.add((state, action, reward, next_state, done))
            else:
                next_state = game.get_state().screen_buffer
                next_state, stacked_frames = stackFrames(stacked_frames, next_state, False)
                memory.add((state, action, reward, next_state, done))
                state = next_state

            tree_idx, loss, abs_errors = TargetNetwork.train(memory, batch_size, DQNetwork())
            memory.batch_update(tree_idx, abs_errors)
            
            if tau > max_tau:
                TargetNetwork.load_weights(DQNetwork())
                tau = 0
                print("Model updated")
            
        if episode % save_frequency == 0:
            TargetNetwork.save_checkpoint(filename=f'Targetscheckpoint{episode}.pth.tar')
            DQNetwork.save_checkpoint(filename=f'DQNcheckpoint{episode}.pth.tar')
    TargetNetwork.save_checkpoint(filename='final.pth.tar')
    DQNetwork.save_checkpoint(filename=f'DQNcheckpoint{episode}.pth.tar')
    game.close()

In [ ]:
if not training:
    DQNetwork.load_checkpoint(filename=trained_model)
    game.init()

    done = False
    
    for i in range(10000):
        game.new_episode()
        state = game.get_state().screen_buffer
        state, stacked_frames = stackFrames(stacked_frames, state, True)
        while not game.is_episode_finished():
            Qs = DQNetwork.predict(state)
            choice = np.argmax(Qs)
            action = possible_actions[int(choice)]

            game.make_action(action)
            done = game.is_episode_finished()
            score = game.get_total_reward()

            if done:
        #         inp = input('Enter anything to replay, or nothing to end')
        #         if not inp:
                break
    #             else:
    #                 game.new_episode()
    #                 state = game.get_state().screen_buffer
    #                 state, stacked_frames = stackFrames(stacked_frames, state, True)
            else:
                next_state = game.get_state().screen_buffer
                next_state, stacked_frames = stackFrames(stacked_frames, next_state, False)
                state = next_state

        score = game.get_total_reward()
        print(f'Score: {score}')
    game.close()

Score: -115.99295043945312
Score: -115.99655151367188
Score: -115.99655151367188
Score: -115.99655151367188
Score: -115.99246215820312
Score: -115.99053955078125
Score: -115.99655151367188
Score: -115.58798217773438
Score: -115.99165344238281
Score: -115.25718688964844
Score: -115.99655151367188
Score: -115.99655151367188
Score: -115.99655151367188
Score: -115.99655151367188
Score: -115.99655151367188
Score: -115.99655151367188
Score: -115.99655151367188
